In [1]:
!pip3 install langchain boto3

In [2]:
import json
from typing import Dict
from langchain import PromptTemplate, SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
from langchain.docstore.document import Document

class MLPLLMEndpoint(SagemakerEndpoint):
    
    class ContentHandler(LLMContentHandler):
        content_type = "application/json"
        accepts = "application/json"
    
        def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
            print('Full Prompt\n', prompt)
            input_str = json.dumps({"inputs": [prompt]})
            return input_str.encode("utf-8")
    
        def transform_output(self, output: bytes) -> str:
            response_json = json.loads(output.read().decode("utf-8"))
            return response_json["generated_text"].strip()

    def __init__(self, model_name, credentials_profile_name, region_name="us-east-1", **kwargs):
        MODEL_ENDPOINT_MAPPING = {
            "llama2-7b": "MG-mlp-llama2-7B-chat-v1-ept"
        }
        endpoint_name = MODEL_ENDPOINT_MAPPING.get(model_name)
        if not endpoint_name:
            raise ValueError(f"Unknown model name: {model_name}")

        content_handler = self.ContentHandler()
        
        super().__init__(endpoint_name=endpoint_name, region_name=region_name, 
                         credentials_profile_name=credentials_profile_name, content_handler=content_handler, **kwargs)

In [3]:
# Initialize the endoint
mlp_llm = MLPLLMEndpoint(
    model_name="llama2-7b",
    credentials_profile_name="wbd-boltcloud-api-dev.GtMlPlatform"
)

In [ ]:
!pip install llama-cpp-python==0.1.48


In [14]:
#from langchain.llms import LlamaCpp
#from langchain.callbacks.manager import CallbackManager
#from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Make sure the model path is correct for your system!
#model_path="/Users/mcreehan/workspace/llama-test/Llama-2-7b-chat-hf/tokenizer.model"
#callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

#llm = LlamaCpp(
#    model_path=model_path,
#    temperature=0.75,
#    max_tokens=2000,
#    top_p=1,
#    callback_manager=callback_manager, 
#    verbose=True, # Verbose is required to pass to the callback manager
 #   n_ctx=2048, n_batch=16, n_gpu_layers=10,
#)

gguf_init_from_file: invalid magic number 050a0e0a
error loading model: llama_model_loader: failed to load model from /Users/mcreehan/workspace/llama-test/Llama-2-7b-chat-hf/tokenizer.model

llama_load_model_from_file: failed to load model


ValidationError: 1 validation error for LlamaCpp
__root__
  Could not load Llama model from path: /Users/mcreehan/workspace/llama-test/Llama-2-7b-chat-hf/tokenizer.model. Received error  (type=value_error)

In [4]:
# Setup your prompt template
prompt_template = """Answer the question below using the context that follows it, only if the information is present in the context.

Question: {question}

{context}

Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [12]:
# Provide your context data
# In this example we supply as simple tabular data in Ascii
context_doc_1 = """
| User_ID | Product_ID | Category   | Purchase_Date | Price | Review_Rating | Review_Text                          |
|---------|------------|------------|---------------|-------|---------------|--------------------------------------|
| 001     | P001       | Electronics| 2022-01-01   | 299.99| 5             | Great product. Totally worth it!     |
| 002     | P002       | Books      | 2022-01-02   | 15.99 | 4             | Informative read, could be shorter.  |
| 003     | P001       | Electronics| 2022-01-02   | 299.99| 2             | Product stopped working after a week.|
| 004     | P003       | Fashion    | 2022-01-03   | 49.99 | 5             | Fits perfectly. Love the design.     |
| ...     | ...        | ...        | ...           | ...   | ...           | 
"""
docs = [
    Document(
        page_content=context_doc_1.replace("\n", " "),
    )
]

In [17]:
# Provide your input query
# In this example, we ask how much one of the Products cost given the Product ID
query = """How much does P001 cost?
"""
# build and run the chain
chain = load_qa_chain(
    mlp_llm,
    prompt=PROMPT,
)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

Full Prompt
 Answer the question below using the context that follows it, only if the information is present in the context.

Question: How much does P001 cost?


 | User_ID | Product_ID | Category   | Purchase_Date | Price | Review_Rating | Review_Text                          | |---------|------------|------------|---------------|-------|---------------|--------------------------------------| | 001     | P001       | Electronics| 2022-01-01   | 299.99| 5             | Great product. Totally worth it!     | | 002     | P002       | Books      | 2022-01-02   | 15.99 | 4             | Informative read, could be shorter.  | | 003     | P001       | Electronics| 2022-01-02   | 299.99| 2             | Product stopped working after a week.| | 004     | P003       | Fashion    | 2022-01-03   | 49.99 | 5             | Fits perfectly. Love the design.     | | ...     | ...        | ...        | ...           | ...   | ...           |  

Answer:


{'output_text': 'The cost of P001 is $299.99.'}

In [18]:
# Provide your input query
# In this example, we ask which User ID might have supplied a negative review
query = """Which user supplied a review which was negative in its sentiment?
"""
# build and run the chain
chain = load_qa_chain(
    mlp_llm,
    prompt=PROMPT,
)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

Full Prompt
 Answer the question below using the context that follows it, only if the information is present in the context.

Question: Which user supplied a review which was negative in its sentiment?


 | User_ID | Product_ID | Category   | Purchase_Date | Price | Review_Rating | Review_Text                          | |---------|------------|------------|---------------|-------|---------------|--------------------------------------| | 001     | P001       | Electronics| 2022-01-01   | 299.99| 5             | Great product. Totally worth it!     | | 002     | P002       | Books      | 2022-01-02   | 15.99 | 4             | Informative read, could be shorter.  | | 003     | P001       | Electronics| 2022-01-02   | 299.99| 2             | Product stopped working after a week.| | 004     | P003       | Fashion    | 2022-01-03   | 49.99 | 5             | Fits perfectly. Love the design.     | | ...     | ...        | ...        | ...           | ...   | ...           |  

Answer:


{'output_text': 'The user who supplied a review with a negative sentiment is User 003, who stated that the product stopped working after a week.'}

In [19]:
# Provide your input query
# In this example, we ask:
query = """Which product id has mixed reviews? 
"""
# build and run the chain
chain = load_qa_chain(
    mlp_llm,
    prompt=PROMPT,
)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

Full Prompt
 Answer the question below using the context that follows it, only if the information is present in the context.

Question: Which product id has mixed reviews? 


 | User_ID | Product_ID | Category   | Purchase_Date | Price | Review_Rating | Review_Text                          | |---------|------------|------------|---------------|-------|---------------|--------------------------------------| | 001     | P001       | Electronics| 2022-01-01   | 299.99| 5             | Great product. Totally worth it!     | | 002     | P002       | Books      | 2022-01-02   | 15.99 | 4             | Informative read, could be shorter.  | | 003     | P001       | Electronics| 2022-01-02   | 299.99| 2             | Product stopped working after a week.| | 004     | P003       | Fashion    | 2022-01-03   | 49.99 | 5             | Fits perfectly. Love the design.     | | ...     | ...        | ...        | ...           | ...   | ...           |  

Answer:


{'output_text': 'The product ID with mixed reviews is P001.'}